# Бартов Олег Борисович
## Решение задачи по предсказанию набора специализаций для вакансии
### 2020-07-27

In [ ]:
#стандартные библиотеки
import pandas as pd
import numpy as np

#для загрузки данных
import json
import gzip

#для очистки htmp-тегов
from bs4 import BeautifulSoup

#прогресс выполнения
from tqdm.notebook import tqdm

#для векторайзеры
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

#пакетная кластеризация KMeans
from sklearn.cluster import MiniBatchKMeans

#для мультилейбл
from sklearn.preprocessing import MultiLabelBinarizer

#бустинг
import catboost

#процедура для загрузки вакансий в словарь
def read_vacancies_part(part):
    with gzip.open(f'vacancies-{part:02}.json.gz', 'r') as fp:
        return json.loads(fp.read())

# №1 Предобработка данных

In [ ]:
%%time

#объявляем стемминг
stemmer = SnowballStemmer('russian')
#запускаем цикл по частям
for part_num in range(1, 11):
    
    #грузим часть
    part = read_vacancies_part(part_num)
    print('Часть '+str(part_num))
    print(f'Всего вакансий в части: {len(part):,d}')
          
    #запускаем цикл для препроцессинга текста
    for k in tqdm(part):
          
        #работаем с наименованием вакансии
        transf_name = part[k]['name']
        #приводим все к нижнему регистру
        transf_name = transf_name.lower()
        #убираем знаки препинания и цифры
        transf_name = re.sub(r"[^a-zа-я]+",' ', transf_name)
        #убираем лишние пробелы
        transf_name = transf_name.strip()
        #делаем стемминг - получается массив
        transf_name = [stemmer.stem(word) for word in transf_name.split(' ')]
        #возвращаем строку из массива обратно
        transf_name = ' '.join(map(str, transf_name))
        #переписываем обработанное наименование вакансии
        part[k]['name'] = transf_name

        
        #теперь работаем с описанием
        #убираем html-теги
        transf_desc = BeautifulSoup(part[k]['description'], "lxml").text
        #приводим все к нижнему регистру
        transf_desc = transf_desc.lower()
        #убираем знаки препинания и цифры
        transf_desc = re.sub(r"[^a-zа-я]+",' ', transf_desc)
        #убираем лишние пробелы
        transf_desc = transf_desc.strip()
        #делаем стемминг - получается массив
        transf_desc = [stemmer.stem(word) for word in transf_desc.split(' ')]        
        #возвращаем строку из массива обратно
        transf_desc = ' '.join(map(str, transf_desc))
        #переписываем обработанное описание вакансии
        part[k]['description'] = transf_desc
        
        
        #работаем с ключевыми навыками вакансии
        transf_skills = part[k]['key_skills']
        #из массива в строку
        transf_skills = ' '.join(map(str, transf_skills))
        #приводим все к нижнему регистру
        transf_skills = transf_skills.lower()
        #убираем лишние пробелы
        transf_skills = transf_skills.strip()
        #делаем стемминг - получается массив
        transf_skills = [stemmer.stem(word) for word in transf_skills.split(' ')]
        #возвращаем строку из массива обратно
        transf_skills = ' '.join(map(str, transf_skills))
        part[k]['key_skills'] = transf_skills
        
        #решаем вопрос с зарплатой - это должен стать один признак
        salary = part[k]['compensation_to']
        if salary is None:
            salary = part[k]['compensation_from']
        if salary is None:
            salary = 0
        part[k]['salary'] = salary
        #убираем лишнее
        part[k].pop('compensation_to')
        part[k].pop('compensation_from')
    
    #записываем часть, которую преобразовали в файл, дальше будем эти файлы объединять
    pd.DataFrame(part).T.to_csv('df_stem'+str(part_num)+'.csv',sep=';')

### объединяем обработанные части в единый датасет

In [ ]:
%%time
df = pd.read_csv('df_stem1.csv',sep=';') #df_part1 #df_clean_morhp1
print(1)
df = df.append(pd.read_csv('df_stem2.csv',sep=';')) #df_part2 #df_clean_morhp2
print(2)
df = df.append(pd.read_csv('df_stem3.csv',sep=';')) #df_part3 #df_clean_morhp3
print(3)
df = df.append(pd.read_csv('df_stem4.csv',sep=';')) #df_part4 #df_clean_morhp4
print(4)
df = df.append(pd.read_csv('df_stem5.csv',sep=';')) #df_part5 #df_clean_morhp5
print(5)
df = df.append(pd.read_csv('df_stem6.csv',sep=';')) #df_part6 #df_clean_morhp6
print(6)
df = df.append(pd.read_csv('df_stem7.csv',sep=';')) #df_part7 #df_clean_morhp7
print(7)
df = df.append(pd.read_csv('df_stem8.csv',sep=';')) #df_part8 #df_clean_morhp8
print(8)
df = df.append(pd.read_csv('df_stem9.csv',sep=';')) #df_part9 #df_clean_morhp9
print(9)
df = df.append(pd.read_csv('df_stem10.csv',sep=';')) #df_part10 #df_clean_morhp10
print(10)

df.rename(columns={'Unnamed: 0': 'vac_id'}, inplace=True) #важная колонка - ключ
df.reset_index(inplace=True) #так как объединение, имеет смысл сбросить индекс
del df['index'] #лишний столбец

#сразу формируем категориальные признаки

#зп
df['sal_clust'] = (df['salary']/1000).astype('int64').astype('category').cat.codes
#валюта
df['currency'].fillna('RUR',inplace=True) #если не указано, пусть будет рубль
df['currency'] = df['currency'].astype('category').cat.codes
#остальные просто преобразуем в категории
df['employer'] = df['employer'].astype('category').cat.codes
df['employment'] = df['employment'].astype('category').cat.codes
df['work_schedule'] = df['work_schedule'].astype('category').cat.codes
df['work_experience'] = df['work_experience'].astype('category').cat.codes

#записываем объединенный датасет файл
df.to_csv('df_stem.csv',sep=';',index=False)
df.head()

# №2 Кластеризация

In [ ]:
%%time
#читаем сохраненный ранее файл (лучше предобработку запускать отдельно из-за проблем с памятью, когда ее мало)
df = pd.read_csv('df_stem.csv',sep=';', parse_dates=['creation_date'])

#дата подачи вакансии еще один категориальный признак - пытаемся поймать сезонность
df['date_feat'] = df['creation_date'].dt.strftime('%Y-%m')
df['date_feat'] = df['date_feat'].astype('category').cat.codes
del df['creation_date']

#смотрим, что прочитали именно то, что нужно
df.head(10)

In [ ]:
#выполняем преобразование, иначе в векторайзер не залезает
df['name'] = df['name'].astype('str')
df['description'] = df['description'].astype('str')
df['key_skills'] = df['key_skills'].astype('str')

### № 2.1 Формируем кластеры по наименованию

In [ ]:
%%time
#tf-idf на мешок слов наименования
vectorizer = TfidfVectorizer(min_df=2)
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1800, max_iter=10, init_size=5400, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_name_1800',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на биграммы наименования
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1500, max_iter=10, init_size=4500, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_2gram_name_1500',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на триграммы наименования
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(3,3))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_3gram_name_2000',minikmeans.labels_)

In [ ]:
%%time
#count на мешок слов наименования
vectorizer = CountVectorizer(min_df=2)
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1500, max_iter=10, init_size=4500, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_count_name_1500',minikmeans.labels_)

In [ ]:
%%time
#count на биграммы наименования
vectorizer = CountVectorizer(min_df=2, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=1500, max_iter=10, init_size=4500, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_count_2gram_name_1500',minikmeans.labels_)

### №2.2 Формируем кластеры по описанию

In [ ]:
%%time
#tf-idf на мешок слов описания
vectorizer = TfidfVectorizer(min_df=10, max_df=100000)
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_description_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на мешок слов другой частотности описания
vectorizer = TfidfVectorizer(min_df=5, max_df=1000)
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=5000, max_iter=10, init_size=15000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_description_s_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на биграммы описания
vectorizer = TfidfVectorizer(max_df=50000,min_df=200, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_2gramm_description_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на триграммы описания
vectorizer = TfidfVectorizer(max_df=50000,min_df=150, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_2gramm_description_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на мешок слов и биграммы описания совместно
vectorizer = TfidfVectorizer(max_df=50000,min_df=200, ngram_range=(1,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_both_description_2000',minikmeans.labels_)

### №2.3 Формируем кластеры по ключевым навыкам

In [ ]:
%%time
#tf-idf на мешок слов ключевых навыков
vectorizer = TfidfVectorizer(min_df=2)
matrix = vectorizer.fit_transform(df['key_skills'])
print(matrix.shape)

#кластеризация
minikmeans = MiniBatchKMeans(n_clusters=2000, max_iter=10, init_size=6000, batch_size=100).fit(matrix)
#сохраняем результат кластеризации во внешний файл
np.save('clust_skills_2000',minikmeans.labels_)

In [ ]:
%%time
#tf-idf на биграммы наименования
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

In [ ]:
%%time
#tf-idf на триграммы наименования
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(3,3))
matrix = vectorizer.fit_transform(df['name'])
print(matrix.shape)

In [ ]:
#tf-idf на мешок слов описания
vectorizer = TfidfVectorizer(min_df=10, max_df=100000)
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

In [ ]:
#tf-idf на мешок слов другой частотности описания
vectorizer = TfidfVectorizer(min_df=5, max_df=1000)
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

In [ ]:
#tf-idf на биграммы описания
vectorizer = TfidfVectorizer(max_df=50000,min_df=200, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

In [ ]:
#tf-idf на триграммы описания
vectorizer = TfidfVectorizer(max_df=50000,min_df=150, ngram_range=(2,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)

In [ ]:
#tf-idf на мешок слов и биграммы описания совместно
vectorizer = TfidfVectorizer(max_df=50000,min_df=200, ngram_range=(1,2))
matrix = vectorizer.fit_transform(df['description'])
print(matrix.shape)